# Постановка задачи
Почта-Банк запустил игровую программу лояльности "Шанс".<br> 
Для того, чтобы ей воспользоваться, необходимо совершить покупки. После этого выбрать желаемый кэшбек от 2 до 12 процентов и бросить игральные кости. Если сумма костей совпадет с желаемым кэшбеков, то клиенту начислится бонус в размере выбранного значения.<br> 
Цель: максимизировать прибыль от кэшбека.<br> 
Задача: найти сумму чисел на двух кубиках, которая является наиболее выигрышной.

# Быстрый ответ
Клиенту банка для максимизации прибыли нужны выбирать 7.

# Решение

## Необходимое условие
Для того, чтобы исследование имело смысл предположим, что виртуальные кубики Почта-Банка честные.

## Математическое решение

По таблице исходов видим, что наиболее частая сумма выпадений-семь.<br> Такая сумма встречается шесть раз. Всего возможных сумм $6*6=36$.<br>
Тогда вероятность получить сумму семь $P= \frac {6}{36}= \frac {1}{6} \approx 0.166$<br>
**Ответ:** Наиболее выигрышная сумма на кубиках **7**

## Программное решение

Допустим, что нам нужно посчитать вероятности других исходов.</br>
А под рукой нет таблицы вариантов, чертить ее долго, а представить в уме сложно. </br>
Для решания такой задачи сгенерируем все возможные варианты possible_amounts.</br>
Создадим датафрейм possible_amounts_df в который запишем информацию о количестве вхождений и вероятности.

In [1]:
import pandas as pd
import numpy as np
import random
from collections import Counter

In [2]:
def generate_df(list_possible_amounts:list, count_options:int)->pd.core.frame.DataFrame:
    df = pd.DataFrame(list(Counter(list_possible_amounts).items()), columns = ['Сумма', 'Количество вхождений'])
    df['Вероятность'] = df['Количество вхождений']/count_options*100
    df = df.sort_values(by='Вероятность', ascending=False).reset_index(drop= True)
    return df

In [3]:
possible_amounts = [i+j for i in range(1,7) for j in range(1,7)]
possible_amounts_df = generate_df(possible_amounts, 36)
possible_amounts_df

,Сумма,Количество вхождений,Вероятность
0,7,6,16.666667
1,6,5,13.888889
2,8,5,13.888889
3,5,4,11.111111
4,9,4,11.111111
5,4,3,8.333333
6,10,3,8.333333
7,3,2,5.555556
8,11,2,5.555556
9,2,1,2.777778


Отсюда видим, что топ-5 наиболее вероятных исодов **{7,6,8,5,9}**.</br>
Прибыль от 6 и 5 процентов меньше чем от 7. Вероятность их появления тоже меньше. Поэтому выведем их из эксперемента.</br>Оставим **7**, как самую вероятную, **8 и 9** как суммы с меньшей вероятностью, но с большей потенциальной прибылью.

### Генерация чисел

Проверим результаты предыдущих вычислений на практике. Также проверим гипотезу- меньшая вероятность выпадения сумм 8 и 9 может компенсироваться дополнительной прибылью связанной с разницей между ними и суммой 7, один и два процента соответсвенно.
Для проверки этой гипотезы создана функция generate_random_sum, которая генерирует список рандомных сумм в колличестве count_random_sum.


In [7]:
def generate_random_sum(count_random_sum:int)->np.array:
    first_value = np.array([random.randint(1,6) for i in range(count_random_sum)])
    second_value = np.array([random.randint(1,6) for i in range(count_random_sum)])
    return first_value+second_value

Далее введем функцию generate_mean_sum, она моделирует генерацию сумм в количестве count_random_sum тысячу раз для статистически значимого результата. Дальше считается среднее количество появлений 7,8,9. Результат работы - список средних количеств появлений.

In [8]:
def generate_mean_sum(count_random_sum:int)->list:
    list_7 = []
    list_8 = []
    list_9 = []
    for i in range(1000):
        dict_sum_value = Counter(generate_random_sum(count_random_sum))
        list_7.append(dict_sum_value[7])
        list_8.append(dict_sum_value[8])
        list_9.append(dict_sum_value[9])
    return [round(np.mean(list_7)), round(np.mean(list_8)), round(np.mean(list_9))]

In [9]:
count_experiment = 10
mean_list_10 = generate_mean_sum(count_experiment)
print(f"Количество эксперементов {count_experiment}\nСреднее число сумм 7-{mean_list_10[0]}\nСреднее число сумм 8-{mean_list_10[1]}\nСреднее число сумм 9-{mean_list_10[2]}")

Количество эксперементов 10
Среднее число сумм 7-2
Среднее число сумм 8-1
Среднее число сумм 9-1


In [10]:
count_experiment = 100
mean_list_100 = generate_mean_sum(count_experiment)
print(f"Количество эксперементов {count_experiment}\nСреднее число сумм 7-{mean_list_100[0]}\nСреднее число сумм 8-{mean_list_100[1]}\nСреднее число сумм 9-{mean_list_100[2]}")

Количество эксперементов 100
Среднее число сумм 7-17
Среднее число сумм 8-14
Среднее число сумм 9-11


In [11]:
count_experiment = 1000
mean_list_1000 = generate_mean_sum(count_experiment)
print(f"Количество эксперементов {count_experiment}\nСреднее число сумм 7-{mean_list_1000[0]}\nСреднее число сумм 8-{mean_list_1000[1]}\nСреднее число сумм 9-{mean_list_1000[2]}")

Количество эксперементов 1000
Среднее число сумм 7-167
Среднее число сумм 8-139
Среднее число сумм 9-111


### Проверка стратегий

Введем функцию testing_strategies, которая принимает результат из функции generate_mean_sum, количество экспериментов, стоимость покупок по клиенту. Выводит информацию о полученном кэшбеке.

In [12]:
def testing_strategies(mean_sums:list, count_experiment:int, cost_purchase:int):
    percents = [0.07, 0.08, 0.09]
    for i, value in enumerate(mean_sums):
        profit = round(cost_purchase*value*percents[i])
        print(f'Предполагаемый кэшбек который получит клиент, если он будет выбирать {round(percents[i]*100)} = {profit}')
    print(f'Стоимсоть покупок {cost_purchase}\nКоличество экспереиментов {count_experiment}')

In [13]:
testing_strategies(mean_list_10, 10, 1000)

Предполагаемый кэшбек который получит клиент, если он будет выбирать 7 = 140
Предполагаемый кэшбек который получит клиент, если он будет выбирать 8 = 80
Предполагаемый кэшбек который получит клиент, если он будет выбирать 9 = 90
Стоимсоть покупок 1000
Количество экспереиментов 10


In [14]:
testing_strategies(mean_list_100, 100, 1000)

Предполагаемый кэшбек который получит клиент, если он будет выбирать 7 = 1190
Предполагаемый кэшбек который получит клиент, если он будет выбирать 8 = 1120
Предполагаемый кэшбек который получит клиент, если он будет выбирать 9 = 990
Стоимсоть покупок 1000
Количество экспереиментов 100


In [15]:
testing_strategies(mean_list_1000, 1000, 1000)

Предполагаемый кэшбек который получит клиент, если он будет выбирать 7 = 11690
Предполагаемый кэшбек который получит клиент, если он будет выбирать 8 = 11120
Предполагаемый кэшбек который получит клиент, если он будет выбирать 9 = 9990
Стоимсоть покупок 1000
Количество экспереиментов 1000


# Выводы
Наиболее выгодная стратегия для клиента при любом количестве покупок выбирать кэшбек 7 процентов.